<a href="https://colab.research.google.com/github/SLVmain/NLP/blob/Les_03/HW_03_nlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Урок 3. Домашняя работа. Embedding word2vec fasttext
Задача поиск похожих по эмбеддингам

Скачиваем датасет (источник): положительные, отрицательные.

или можно через ноутбук

!wget https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv
!wget https://www.dropbox.com/s/r6u59ljhhjdg6j0/negative.csv

что надо сделать
1. объединить в одну выборку
2. на основе word2vec/fasttext/glove/слоя Embedding реализовать метод поиска ближайших твитов
(на вход метода должен приходить запрос (какой-то твит, вопрос) и количество вариантов вывода к примеру 5-ть, ваш метод должен возвращать 5-ть ближайших твитов к этому запросу)
3. Проверить насколько хорошо работают подходы

In [ ]:
import pandas as pd
import numpy as np
import re

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import warnings 
warnings.filterwarnings('ignore')
import os
import time

from sklearn.metrics import *
from sklearn.model_selection import train_test_split

from collections import Counter

import nltk
import html

from nltk.tokenize import word_tokenize, wordpunct_tokenize
from nltk import tokenize as tknz
from nltk.corpus import stopwords

nltk.download('genesis')
nltk.download('punkt')

[nltk_data] Downloading package genesis to /root/nltk_data...
[nltk_data]   Unzipping corpora/genesis.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', 800)

In [ ]:
!pip install pymorphy2

In [ ]:
!pip install stop_words

In [ ]:
!pip install annoy

In [ ]:
import string
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
import annoy
from gensim.models import Word2Vec, FastText
import pickle
from tqdm import tqdm_notebook

In [ ]:
!wget https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv
!wget https://www.dropbox.com/s/r6u59ljhhjdg6j0/negative.csv

In [ ]:
positive = pd.read_csv('positive.csv', sep=';', usecols=[3], names=['text'])
negative = pd.read_csv('negative.csv', sep=';', usecols=[3], names=['text'])
df = positive.append(negative)
df.head()

,text
0,"@first_timee хоть я и школота, но поверь, у нас то же самое :D общество профилирующий предмет типа)"
1,"Да, все-таки он немного похож на него. Но мой мальчик все равно лучше:D"
2,RT @KatiaCheh: Ну ты идиотка) я испугалась за тебя!!!
3,"RT @digger2912: ""Кто то в углу сидит и погибает от голода, а мы ещё 2 порции взяли, хотя уже и так жрать не хотим"" :DD http://t.co/GqG6iuE2…"
4,"@irina_dyshkant Вот что значит страшилка :D\nНо блин,посмотрев все части,у тебя создастся ощущение,что авторы курили что-то :D"


In [ ]:
df.describe()

,text
count,226834
unique,217440
top,"Офигенный день!\nдень позитива)\nбегал как идиот целый день!\nтанцы офигенны, хоть я и ракал но мне очень понравилось!"
freq,156


In [ ]:
from string import punctuation
punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [ ]:
from string import whitespace
whitespace

' \t\n\r\x0b\x0c'

In [ ]:
def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

In [ ]:
def remove_nick(input_txt):
  r = re.findall("@[\w]*", input_txt)
  for i in r:
    input_txt = re.sub(i, '', input_txt)

  return input_txt

In [ ]:
df.text.head()

0                                             @first_timee хоть я и школота, но поверь, у нас то же самое :D общество профилирующий предмет типа)
1                                                                         Да, все-таки он немного похож на него. Но мой мальчик все равно лучше:D
2                                                                                           RT @KatiaCheh: Ну ты идиотка) я испугалась за тебя!!!
3    RT @digger2912: "Кто то в углу сидит и погибает от голода, а мы ещё 2 порции взяли, хотя уже и так жрать не хотим" :DD http://t.co/GqG6iuE2…
4                  @irina_dyshkant Вот что значит страшилка :D\nНо блин,посмотрев все части,у тебя создастся ощущение,что авторы курили что-то :D
Name: text, dtype: object

In [ ]:
sw = set(get_stop_words("ru"))
exclude = set(string.punctuation + string.whitespace)
morpher = MorphAnalyzer()

def preprocess_line(text): 
    text = "".join(i.strip() for i in text if i not in exclude).split() #,punc, /t

    text = [remove_nick(i) for i in text]
    text = [remove_urls(i) for i in text]
    text = [x.lower() for x in text]
    
    text = [x.replace('[^\w\s]','') for x in text]
    
    text = [morpher.parse(i)[0].normal_form for i in text] #lemma
    
    text = [i for i in text if i not in sw and i != ""] #stopw
    return text

In [ ]:
sentenses = [preprocess_line(t) for t in tqdm_notebook(df['text'])]

  0%|          | 0/226834 [00:00<?, ?it/s]

# Модели Word2Vec, Fasttext

In [ ]:
import gensim.downloader as api

In [ ]:
# обучим модели на наших твитах

modelW2V = Word2Vec(sentences=sentenses, size=300, window=5, min_count=1)

modelFT = FastText(sentences=sentenses, size=300, min_count=1, window=5, workers=8)

In [ ]:
# посчитаем векторы наших твитов
index_w2v = annoy.AnnoyIndex(300 ,'angular')
index_ft = annoy.AnnoyIndex(300 ,'angular')

for i, sent in enumerate(sentenses):

    res1 = [modelW2V.wv[w] for w in sent if w in modelW2V.wv]
    v_w2v = sum(res1) / len(res1) if res1 else np.zeros(300)

    res2 = [modelFT.wv[w] for w in sent if w in modelFT.wv]
    v_ft = sum(res2) / len(res2) if res2 else np.zeros(300)


    index_w2v.add_item(i, v_w2v)
    index_ft.add_item(i, v_ft)



index_w2v.build(10)
index_ft.build(10)

# index_w2v, index_ft

True

In [ ]:
def get_response(question, index, model):
    question = preprocess_line(question)
    vector = np.zeros(300)
    norm = 0
    for word in question:
        if word in model.wv:
            vector += model.wv[word]
            norm += 1
    if norm > 0:
        vector = vector / norm
    answers = index.get_nns_by_vector(vector, 5, )
    return [df['text'].iloc[i] for i in answers]

def res_print(question, index, model):
  print(f'Исходный твит: {question}')
  answers2 = get_response(question, index, model)
  for i, item in enumerate(answers2):
    print(f'{i+1} ближайший твит \n {item}')

In [ ]:
test_tw = df.text[1]

In [ ]:
res_print(test_tw, index_ft, modelFT)

Исходный твит: 1       Да, все-таки он немного похож на него. Но мой мальчик все равно лучше:D
1    Коллеги сидят рубятся в Urban terror, а я из-за долбанной винды не могу :(
Name: text, dtype: object
1 ближайший твит 
 @borisova_diana та пофиг, у меня все равно денег нет(((9(( надо еще мальчишек замутить, а то мы же все шлюшки 
2 ближайший твит 
 @Reno_oppa @maru_gin у меня так скоро отп не останется хд
3 ближайший твит 
 @abakanskiy тогда это уже не просо пиво, это уже окрошка ;))
4 ближайший твит 
 У нас завелся новый жирный троль))) Ура товарищи!!!
5 ближайший твит 
 RT @dimcooper: Собрался апроксимировать, а потом понял, что всё гораздо проще. Просто апроксиматор зачесался (((


In [ ]:
res_print(test_tw, index_w2v, modelW2V)

Исходный твит: 1       Да, все-таки он немного похож на него. Но мой мальчик все равно лучше:D
1    Коллеги сидят рубятся в Urban terror, а я из-за долбанной винды не могу :(
Name: text, dtype: object
1 ближайший твит 
 Первый раз готовила спагетти,у меня получилось и я очень рада) http://t.co/xaubtn3BfA
2 ближайший твит 
 Интересные вещи творились тогда)) http://t.co/jK7W5Lr9l0
3 ближайший твит 
 Очень крутые ребята! :), только качество не очень #CityLights http://t.co/L2CQR3JUb0
4 ближайший твит 
 RT @Crazy_Mars_04: @99Nastya0804 пхаха это отдельная история:D
5 ближайший твит 
 RT @abnepo: Пришел к Френку. Какой же он красивый!..)))


сложно оценить, что работает лучше. на мой взгляд, лучше обучать нейросеть на этих векторах, а не пользоваться similarity и annoi